In [1]:
from cgeniepy.ecology import EcoModel
import pandas as pd
import matplotlib as mpl
mpl.rcParams.update(mpl.rcParamsDefault)
from cgeniepy.array import GenieArray
import numpy as np

foram = ["bn", "bs", "sn", "ss"]

mod_paths = [
              "../model/muffin.CBE.worlg4.BASESFeTDTL.SPIN",
              "../model/muffin.CBE.GIteiiva.BASESFeTDTL_rb.SPIN",
            "../model/muffin.CBE.worlg4.BASESFeTDTL.historical",
             "../model/muffin.CBE.worlg4.BASESFeTDTL.2100.1p5deg",
             "../model/muffin.CBE.worlg4.BASESFeTDTL.2100.2deg",
             "../model/muffin.CBE.worlg4.BASESFeTDTL.2100.3deg",
             "../model/muffin.CBE.worlg4.BASESFeTDTL.2100.4deg",
             "../model/muffin.CBE.worlg4.BASESFeTDTL.2.5xCO2"
             ]

export_files= ["../model/model_drived/pi_foramecogenie.csv",
               "../model/model_drived/lgm_foramecogenie.csv",
               "../model/model_drived/historical_foramecogenie.csv",
               "../model/model_drived/future1p5_foramecogenie.csv",
                "../model/model_drived/future2_foramecogenie.csv",
                "../model/model_drived/future3_foramecogenie.csv",
                "../model/model_drived/future4_foramecogenie.csv",
               "../model/model_drived/2.5xCO2_foramecogenie.csv"
               ]

def get_abundance(model, foram):
    """
    convert biomass to absolute abundance 
    """
    ## mmol C/m3 => ind/m3

    ## biovolume in um3
    biovolume = model.eco_pars().query(f"PFT == 'foram_{foram}'")['volume']
    ## biomass density in g C/um3
    density = 8.9E-14
    

    ## get biomass per individual
    biomass_per_ind =  biovolume * density
    
    ## get biomass field
    biomass = model.get_foram(foram).isel(time=-1).array

    ## filter those with less than 1 cell/m3
    biomass = biomass.where(biomass > biomass_per_ind.values, biomass, 0)

    ## convert biomass to abundance
    abundance = biomass / biomass_per_ind.values    

    return abundance

def get_rel_abun(model, foram):

    all_foram = ['bn','bs','sn','ss']
    total_abundance = np.zeros([36, 36])
    ## get total foram abundance
    for f in all_foram:
        total_abundance += get_abundance(model, f)

    ## get relative abundance
    ra = get_abundance(model, foram) / total_abundance

    ## return GenieArray
    genie_array = GenieArray()
    genie_array.array = ra
    return genie_array

for path, csv_name in zip(mod_paths, export_files):
    mod = EcoModel(path)
    
    bn_c = get_rel_abun(mod, "bn").array.values.ravel()
    bs_c = get_rel_abun(mod, "bs").array.values.ravel()
    sn_c = get_rel_abun(mod, "sn").array.values.ravel()
    ss_c = get_rel_abun(mod, "ss").array.values.ravel()    

    sst = mod.get_var("ocn_sur_temp").isel(time=-1).array.values.ravel()
    sal = mod.get_var("ocn_sur_sal").isel(time=-1).array.values.ravel()
    po4 = mod.get_var("ocn_sur_PO4").isel(time=-1).array.values.ravel()
    light = mod.get_var("phys_fxsw").isel(time=-1).array.values.ravel()
    chl_total = mod.get_var("eco2D_Plankton_C_Total").isel(time=-1).array.values.ravel()
    
    fe = mod.get_var("ocn_sur_TDFe").isel(time=-1).array.values.ravel()
    mld = mod.get_var("phys_MLD").isel(time=-1).array.values.ravel()

    # masked!
    #random_var = mod.select_var("ocn_sur_temp").isel(time=-1).array.values
    #mask = np.where(np.isnan(random_var), np.nan, 1)
    ## masked lat
    #masked_lat = lat_array * mask

    #phys_w = mod.select_var("phys_w").isel(time=-1, zt=1).array
    #upwelling_discrete = np.where(phys_w >0, 1, 0)

    # temperature gradient between 283 m and 80 m
    #therm_gradient = mod.get_var("phys_w").isel(time=-1, zt=3).array - mod.get_var("phys_w").isel(time=-1, zt=1).array

    ## create data frame
    df = pd.DataFrame({"bn": bn_c, "bs": bs_c, "sn": sn_c, "ss": ss_c,                    
                       "sst": sst, "sal": sal, "po4": po4, "light": light,
                       "chl_total": chl_total,"fe": fe, "mld": mld,
                       #"therm_gradient": therm_gradient,
                       #"upwelling": upwelling_discrete.ravel(),
                       #"masked_lat": masked_lat.ravel()
                       })    

    # drop na
    df = df.dropna()
    #df = df.query('bn > 0 & bs > 0 & sn > 0 & ss > 0')
    #df = df.query('bn < 1 & bs < 1 & sn < 1 & ss < 1')
    df.to_csv(csv_name)

<frozen importlib._bootstrap>:241: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


>>> No gemflag is provided, use default gemflags: [biogem, ecogem]
>>> No gemflag is provided, use default gemflags: [biogem, ecogem]
>>> No gemflag is provided, use default gemflags: [biogem, ecogem]
>>> No gemflag is provided, use default gemflags: [biogem, ecogem]
>>> No gemflag is provided, use default gemflags: [biogem, ecogem]
>>> No gemflag is provided, use default gemflags: [biogem, ecogem]
>>> No gemflag is provided, use default gemflags: [biogem, ecogem]
>>> No gemflag is provided, use default gemflags: [biogem, ecogem]


## Next cell is similar but modifies the function for Ying et al. GMD model output

In [ ]:
from cgeniepy.ecology import EcoModel
from cgeniepy.array import GenieArray
import numpy as np
import pandas as pd

def get_abundance(model, foram):
    """
    convert biomass to absolute abundance 
    """
    ## mmol C/m3 => ind/m3

    ## biovolume in um3
    ## note the name convention is changed in the new version
    biovolume = model.eco_pars().query(f"PFT == '{foram}_foram'")['volume']
    
    ## biomass density in g C/um3
    density = 8.9E-14    

    ## get biomass per individual
    biomass_per_ind =  biovolume * density
    
    ## get biomass field
    biomass = model.get_foram(foram).mean(dim=['time']).array

    ## filter those with less than 1 cell/m3
    #biomass = biomass.where(biomass > biomass_per_ind.values, biomass, 0)

    ## convert biomass to abundance
    abundance = biomass / biomass_per_ind.values    

    return abundance

def get_rel_abun(model, foram):

    all_foram = ['bn','bs','sn','ss']
    total_abundance = np.zeros([36, 36])
    ## get total foram abundance
    for f in all_foram:
        total_abundance += get_abundance(model, f)

    ## get relative abundance
    ra = get_abundance(model, foram) / total_abundance

    ## return GenieArray
    genie_array = GenieArray()
    genie_array.array = ra
    return genie_array

path = "/Users/yingrui/Downloads/muffin.CB.worlg4.BASESFeTDTL.FORAM.monthly.SPIN/"
mod = EcoModel(path)

bn_c = get_rel_abun(mod, "bn").array.values.ravel()
bs_c = get_rel_abun(mod, "bs").array.values.ravel()
sn_c = get_rel_abun(mod, "sn").array.values.ravel()
ss_c = get_rel_abun(mod, "ss").array.values.ravel()    

sst = mod.get_var("ocn_sur_temp").isel(time=-1).array.values.ravel()
sal = mod.get_var("ocn_sur_sal").isel(time=-1).array.values.ravel()
po4 = mod.get_var("ocn_sur_PO4").isel(time=-1).array.values.ravel()
light = mod.get_var("phys_fxsw").isel(time=-1).array.values.ravel()
chl_total = mod.get_var("eco2D_Plankton_C_Total").isel(time=-1).array.values.ravel()

fe = mod.get_var("ocn_sur_TDFe").isel(time=-1).array.values.ravel()
mld = mod.get_var("phys_MLD").isel(time=-1).array.values.ravel()

## create data frame
df = pd.DataFrame({"bn": bn_c, "bs": bs_c, "sn": sn_c, "ss": ss_c,                    
                    "sst": sst, "sal": sal, "po4": po4, "light": light,
                    "chl_total": chl_total,"fe": fe, "mld": mld,                    
                    })    

# drop na
df = df.dropna()
#df = df.query('bn > 0 & bs > 0 & sn > 0 & ss > 0')
#df = df.query('bn < 1 & bs < 1 & sn < 1 & ss < 1')
df.to_csv("../model/model_drived/piold_foramecogenie.csv")

No gemflag is provided, assuming the model includes biogem and ecogem
